In [1]:
%matplotlib inline 
import matplotlib.pyplot as plt 
import numpy as np 
import pymc3 as pm 
import pandas as pd

data = pd.read_csv('radon.csv')
county_names = data.county.unique() 
countyidx = data['county_code'].values 
n_counties = len(data.county.unique())

In [2]:
data[['county', 'log_radon', 'floor']].head()

,county,log_radon,floor
0,AITKIN,0.832909,1
1,AITKIN,0.832909,0
2,AITKIN,1.098612,0
3,AITKIN,0.095310,0
4,ANOKA,1.163151,0


In [3]:
### Question: whether basement increases radon in house

In [ ]:
### Probabilistic programming...

indiv_traces = {}
for county_name in county_names:
    # Select subset of data belonging to county
    c_data = data.ix[data.county == county_name]
    c_log_radon = c_data.log_radon
    c_floor_measure = c_data.floor.values
    
    with pm.Model() as individual_model:
        # Intercept prior (variance == sd**2)
        a = pm.Normal('alpha', mu=0, sd=100**2)
        # Slope prior
        b = pm.Normal('beta', mu=0, sd=100**2)
    
        # Model error prior
        eps = pm.Uniform('eps', lower=0, upper=100)
    
        # Linear model
        radon_est = a + b * c_floor_measure
    
        # Data likelihood
        radon_like = pm.Normal('radon_like', mu=radon_est, sd=eps, observed=c_log_radon)

        # Inference button (TM)!
        step = pm.NUTS()
        trace = pm.sample(2000, step=step, progressbar=False)
    
    # keep trace for later analysis
    indiv_traces[county_name] = trace

ERROR (theano.gof.opt): SeqOptimizer apply <theano.tensor.opt.FusionOptimizer object at 0x7fd2caf23c50>
ERROR:theano.gof.opt:SeqOptimizer apply <theano.tensor.opt.FusionOptimizer object at 0x7fd2caf23c50>
ERROR (theano.gof.opt): Traceback:
ERROR:theano.gof.opt:Traceback:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/datascience/bin/python/anaconda3/lib/python3.4/site-packages/theano/gof/opt.py", line 196, in apply
    sub_prof = optimizer.optimize(fgraph)
  File "/datascience/bin/python/anaconda3/lib/python3.4/site-packages/theano/gof/opt.py", line 82, in optimize
    ret = self.apply(fgraph, *args, **kwargs)
  File "/datascience/bin/python/anaconda3/lib/python3.4/site-packages/theano/tensor/opt.py", line 5499, in apply
    new_outputs = self.optimizer(node)
  File "/datascience/bin/python/anaconda3/lib/python3.4/site-packages/theano/tensor/opt.py", line 5434, in local_fuse
    n = OP(C)(*inputs).owner
  File "/datascience/bin/python/anaconda3/lib/python3.4/site-p

In [ ]:
### Hierarchical Model --- makes group parameters that consider counties
# ... as having underlying similarity. These distributions are used to influence
# ... distribution of each county's 'a' and 'b'. 

with pm.Model() as hierarchical_model:
    # Hyperpriors for group nodes
    mu_a = pm.Normal('mu_alpha', mu=0., sd=100**2)
    sigma_a = pm.Uniform('sigma_alpha', lower=0, upper=100)
    mu_b = pm.Normal('mu_beta', mu=0., sd=100**2)
    sigma_b = pm.Uniform('sigma_beta', lower=0, upper=100)
    
    # Intercept for each county, distributed around group mean mu_a
    # Above we just set mu and sd to a fixed value while here we
    # plug in a common group distribution for all a and b (which are
    # vectors of length n_counties).
    a = pm.Normal('alpha', mu=mu_a, sd=sigma_a, shape=n_counties)
    # Intercept for each county, distributed around group mean mu_a
    b = pm.Normal('beta', mu=mu_b, sd=sigma_b, shape=n_counties)
    
    # Model error
    eps = pm.Uniform('eps', lower=0, upper=100)
    
    # Model prediction of radon level
    # a[county_idx] translates to a[0, 0, 0, 1, 1, ...],
    # we thus link multiple household measures of a county
    # to its coefficients.
    radon_est = a[county_idx] + b[county_idx] * data.floor.values
    
    # Data likelihood
    radon_like = pm.Normal('radon_like', mu=radon_est, sd=eps, observed=data.log_radon)

In [ ]:
# Inference button (TM)!
with hierarchical_model:
    start = pm.find_MAP()
    step = pm.NUTS(scaling=start)
    hierarchical_trace = pm.sample(2000, step, start=start, progressbar=False)

In [ ]:
# Plotting the hierarchical model trace -its found values- from 500 iterations 
# onwards (right side plot) 
# and its accumulated marginal values (left side plot) 
pm.traceplot(hierarchical_trace[500:])